In [1]:
import torch
import torch.nn as nn
from counter import count_flops_params
from torchvision.models import resnet18, resnet50

### Refactor counter

1. Hard to debug thop. Make verbose easier read.
2. Each layer may be caled multiple times. Add #Calls to represent multiple calls.

In [2]:
from thop import profile
import torch
# args = dict(inputs=(x, ),
#                 custom_ops=custom_ops_for_efficient,
#                 verbose=False)
#     flops, params = thop.profile(model, **args)
    
x = torch.randn(1, 3, 224, 224)
flops, params = profile(resnet50(), inputs=(x,))
print(flops, params)

in_features 2048 out_features 1000
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchvision.models.resnet.Bottleneck'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torchvision.models.resnet.ResNet'>. Treat it as zero Macs and zero Params.
4111514624.0 25557032.0


In [6]:
flops, params, results = count_flops_params(resnet50(), (1, 3, 224, 224), verbose=True, show_data_size=False)
print(flops, params)

in_features 2048 out_features 1000
----------  -----------------------  --------  --------------------  ----------  ----------  
     Index                     Name    Module           Weight Size       FLOPs     #Params  
         0                    conv1    Conv2d         (64, 3, 7, 7)   118013952        9408  
         1           layer1.0.conv1    Conv2d        (64, 64, 1, 1)    12845056        4096  
         2           layer1.0.conv2    Conv2d        (64, 64, 3, 3)   115605504       36864  
         3           layer1.0.conv3    Conv2d       (256, 64, 1, 1)    51380224       16384  
         4    layer1.0.downsample.0    Conv2d       (256, 64, 1, 1)    51380224       16384  
         5           layer1.1.conv1    Conv2d       (64, 256, 1, 1)    51380224       16384  
         6           layer1.1.conv2    Conv2d        (64, 64, 3, 3)   115605504       36864  
         7           layer1.1.conv3    Conv2d       (256, 64, 1, 1)    51380224       16384  
         8           laye

In [9]:
class Model2(nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        self.conv = nn.Conv2d(3, 5, 1, 1)
        self.conv2 = nn.Conv2d(5, 5, 1, 1)
    def forward(self, x):
        x = self.conv(x)
        for i in range(3):
            x = self.conv2(x)
        return x

flops, params, results = count_flops_params(Model2(), (1, 3, 224, 224), verbose=True, show_data_size=False)
print(flops, params)

----------  -------  --------  --------------  ----------  ----------  ----------  
     Index     Name    Module     Weight Size       FLOPs     #Params      #Calls  
         0     conv    Conv2d    (5, 3, 1, 1)      752640          20           1  
         1    conv2    Conv2d    (5, 5, 1, 1)     1254400          30           3  
----------  -------  --------  --------------  ----------  ----------  ----------  
FLOPs total: 4515840 
#Params total: 110 

4515840 110


In [10]:
print(results)

[{'name': 'conv', 'module_type': 'Conv2d', 'flops': 752640, 'params': 20, 'num_calls': 1, 'weight_size': (5, 3, 1, 1), 'input_sizes': (1, 3, 224, 224), 'output_sizes': (1, 5, 224, 224)}, {'name': 'conv2', 'module_type': 'Conv2d', 'flops': 1254400, 'params': 30, 'num_calls': 3, 'weight_size': (5, 5, 1, 1), 'input_sizes': (1, 5, 224, 224), 'output_sizes': (1, 5, 224, 224)}]
